# Generating repository of two-qubit cliffords

In [203]:
import numpy as np
import pickle
from numpy.random import choice
from random import choice, uniform
from routines_aux import style
from tqdm.auto import tqdm

In [2]:
# generate each of the two qubit cliffords
# populate dictionary mapping between raw index of 11520 gates to its C2 composition

dictClif = {}

def lst_gen(lst):
    for item in lst:
        yield item
        
# total number of gates
index_gen = lst_gen(np.arange(11520))

singleClass = []
for i in range(24):
    for j in range(24):
        tab = allGens()
        C1(tab, 0, i)
        C1(tab, 1, j)
        singleClass.append(tab)
        dictClif[next(index_gen)] = ['single', i, j]
        
cnotClass = []
for i in range(24):
    for j in range(24):
        for k in S1_ind:
            for l in S1y_ind:
                tab = allGens()
                C1(tab, 0, i)
                C1(tab, 1, j)
                CZ(tab, 0, 1)
                C1(tab, 0, k)
                C1(tab, 1, l)
                cnotClass.append(tab)
                dictClif[next(index_gen)] = ['cnot', i, j, k, l]
                
iswapClass = []
for i in range(24):
    for j in range(24):
        for k in S1y_ind:
            for l in S1x_ind:
                tab = allGens()
                C1(tab, 0, i)
                C1(tab, 1, j)
                CZ(tab, 0, 1)
                C1(tab, 0, C1_names.index('Y2'))
                C1(tab, 1, C1_names.index('-X2'))
                CZ(tab, 0, 1)
                C1(tab, 0, k)
                C1(tab, 1, l)
                iswapClass.append(tab)
                dictClif[next(index_gen)] = ['iswap', i, j, k, l]
                
swapClass = []
for i in range(24):
    for j in range(24):
        tab = allGens()
        C1(tab, 0, i)
        C1(tab, 1, j)
        CZ(tab, 0, 1)
        C1(tab, 0, C1_names.index('-Y2'))
        C1(tab, 1, C1_names.index('Y2'))
        CZ(tab, 0, 1)
        C1(tab, 0, C1_names.index('Y2'))
        C1(tab, 1, C1_names.index('-Y2'))
        CZ(tab, 0, 1)
        C1(tab, 1, C1_names.index('Y2'))
        swapClass.append(tab)
        dictClif[next(index_gen)] = ['swap', i, j]
        
allGates = np.concatenate((singleClass, cnotClass, iswapClass, swapClass))

In [10]:
# save locally

outfile = open('allGates', 'wb')
pickle.dump(allGates, outfile)
outfile.close()

outfile = open('dictClif', 'wb')
pickle.dump(dictClif, outfile)
outfile.close()

# A few sanity checks

In [8]:
def gfunc(x1, z1, x2, z2):
    if ((x1 != 0) and (x1 != 1)) or ((z1 != 0) and (z1 != 1)) or ((x2 != 0) and (x2 != 1)) or ((z2 != 0) and (z2 != 1)):
        raise Exception('invalid inputs')
    if (x1 == 0) and (z1 == 0):
        return 0
    if (x1 == 1) and (z1 == 1):
        return (z2 - x2)
    if (x1 == 1) and (z1 == 0):
        return z2*(2*x2-1)
    if (x1 == 0) and (z1 == 1):
        return x2*(1-2*z2)

In [9]:
for i in range(len(C1_trans)):
    """
    for each of the 24 single qubit Cliffords
    get phase of multiplying X and Z gates via nonlinear phase update rowsum rule in Aaronson-Gottesman
    verify that this phase is indeed the phase of Y
    """
    phase = 1 + 2*C1_trans[i][0,2] + 2*C1_trans[i][2,2]
    phase += gfunc(C1_trans[i][0,0], C1_trans[i][0,1], C1_trans[i][2,0], C1_trans[i][2,1])
    phase = (phase % 4) // 2
    if phase != C1_trans[i][1,2]:
        raise Exception
    print(C1_names[i], phase)

I 0
X 1
Y 0
Y, X 1
X2, Y2 0
X2, -Y2 1
-X2, Y2 1
-X2, -Y2 0
Y2, X2 0
Y2, -X2 1
-Y2, X2 0
-Y2, -X2 1
X2 0
-X2 1
Y2 0
-Y2 0
-X2, Y2, X2 1
-X2, -Y2, X2 0
X, Y2 1
X, -Y2 1
Y, X2 0
Y, -X2 1
X2, Y2, X2 0
-X2, Y2, -X2 1


In [12]:
ref = allGens()
for ind in range(len(allGates)):
    """
    for each of the 11520 Clifford two qubit gates
    verify that the transformation on each stabiliser can be generated from the four basis stabilisers
    0001 (IZ), 0010 (ZI), 0100 (IX), 1000 (XI), modulo phase
    phase bit is nolinear and can not be deduced from these basis elemets
    """
    for row in range(16):
        val = (ref[row,0] * allGates[ind][8,:-1] + 
               ref[row,1] * allGates[ind][4,:-1] +
               ref[row,2] * allGates[ind][2,:-1] +
               ref[row,3] * allGates[ind][1,:-1])
        if np.sum((val - allGates[ind][row,:-1]) % 2) != 0:
            raise Exception('bad')
print('success')

success


In [13]:
for i in range(10000):
    """
    check that there are no duplicate gates (probabilistically)
    """
    a = choice(np.arange(len(allGates)))
    b = choice(np.arange(len(allGates)))
    if a != b:
        if np.array_equal(allGates[a], allGates[b]):
            raise Exception('bad')
print('success')

success


## Recover 4x4 unitary gate representation from GF2 action on stabilisers

In [3]:
infile = open('allGates', 'rb')
allGates = pickle.load(infile)
infile.close()
print(len(allGates))

infile = open('dictClif', 'rb')
dictClif = pickle.load(infile)
infile.close()
print(len(dictClif))

11520
11520


In [6]:
for gate_ind in tqdm(range(10944,11520)):
    """
    verify that gate as described by the decomp in dictClif matches that of the reconstructed unitary gate 
    
    single: range(0, 576)
    cnot: range(576, 5760)
    iswap: range(5760,10944)
    swap: range(10944,11520) 
    """
    decomp = dictClif[gate_ind]
    gate = decompToUni(decomp)

    pre = allGens()
    post = allGates[gate_ind]

    for i in range(len(pre)):
        stab_pre = stabBinToUni_2(pre[i])
        stab_post = stabBinToUni_2(post[i])
        if not np.array_equal(stab_post, np.round(gate @ stab_pre @ gate.H, 0)):
            print(i)
            raise Exception()

# Characterising gates

For the CZ gate

We can verify that for the basis elements

$$ X \otimes I \rightarrow X \otimes X $$
$$ I \otimes X \rightarrow Z \otimes X $$
$$ Z \otimes I \rightarrow Z \otimes I $$
$$ I \otimes Z \rightarrow I \otimes Z $$

This implies (and is verified)

$$ X \otimes X \rightarrow Y \otimes Y $$
$$ Z \otimes Z \rightarrow Z \otimes Z $$
$$ Y \otimes I \rightarrow Y \otimes Z $$
$$ I \otimes Y \rightarrow Z \otimes Y $$

$$ X \otimes Y \rightarrow -Y \otimes X $$
$$ Y \otimes X \rightarrow -X \otimes Y $$

We derive a tableau rule, for CZ between qubits a and b (a and b have symmetric roles uder this gate)

$$ z_a \rightarrow z_a \oplus x_b $$
$$ z_b \rightarrow z_b \oplus x_a $$
$$ r \rightarrow r \oplus 1 $$ iff XY or YX
$$ r \rightarrow r $$ otherwise

In [8]:
# USEFUL
# control-Z is symmetric between the control and target bit

print(cz)
print()

# apply(cz, np.kron(paulix, np.identity(2)), np.kron(paulix, pauliz));
# apply(cz, np.kron(np.identity(2), paulix), np.kron(pauliz, paulix));
# apply(cz, np.kron(pauliz, np.identity(2)), np.kron(pauliz, np.identity(2)));
# apply(cz, np.kron(np.identity(2), pauliz), np.kron(np.identity(2), pauliz));

# apply(cz, np.kron(np.identity(2), pauliy), np.kron(pauliz, pauliy));
# apply(cz, np.kron(pauliy, np.identity(2)), np.kron(pauliy, pauliz));
# apply(cz, np.kron(pauliy, pauliy), np.kron(paulix, paulix));

apply(cz, np.kron(paulix, paulix), np.kron(pauliy, pauliy));
# apply(cz, np.kron(pauliz, pauliz), np.kron(pauliz, pauliz));

# apply(cz, np.kron(paulix, pauliz), np.kron(paulix, np.identity(2)));
# apply(cz, np.kron(pauliy, pauliz), np.kron(pauliy, np.identity(2)));

apply(cz, np.kron(paulix, pauliy), -np.kron(pauliy, paulix));
apply(cz, np.kron(pauliy, paulix), -np.kron(paulix, pauliy));

[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]

True
True
True


In [7]:
# verify action of swap class of two-qubit gates
# in implementation, expressed at composition of CZ and single qubits

apply(swap, np.kron(paulix, np.identity(2)), np.kron(np.identity(2), paulix));
apply(swap, np.kron(np.identity(2), paulix), np.kron(paulix, np.identity(2)));
apply(swap, np.kron(pauliz, np.identity(2)), np.kron(np.identity(2), pauliz));
apply(swap, np.kron(np.identity(2), pauliz), np.kron(pauliz, np.identity(2)));

apply(swap, np.kron(np.identity(2), pauliy), np.kron(pauliz, pauliy));
apply(swap, np.kron(paulix, paulix), np.kron(paulix, np.identity(2)));
apply(swap, np.kron(pauliy, pauliy), -np.kron(paulix, pauliz));
apply(swap, np.kron(pauliy, paulix), np.kron(pauliy, np.identity(2)));

True
True
True
True
False
[[0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]]
False
[[0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]
False
[[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]]
False
[[0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]]


In [10]:
# verify action of cnot class of two-qubit gates
# in implementation, expressed at composition of CZ and single qubits

# apply(cnot, np.kron(paulix, np.identity(2)), np.kron(paulix, paulix));
# apply(cnot, np.kron(np.identity(2), paulix), np.kron(np.identity(2), paulix));
# apply(cnot, np.kron(pauliz, np.identity(2)), np.kron(pauliz, np.identity(2)));
# apply(cnot, np.kron(np.identity(2), pauliz), np.kron(pauliz, pauliz));

# apply(cnot, np.kron(np.identity(2), pauliy), np.kron(pauliz, pauliy));
# apply(cnot, np.kron(paulix, paulix), np.kron(paulix, np.identity(2)));
# apply(cnot, np.kron(pauliy, pauliy), -np.kron(paulix, pauliz));
# apply(cnot, np.kron(pauliy, paulix), np.kron(pauliy, np.identity(2)));

apply(cnot, np.kron(np.identity(2), pauliz), np.kron(pauliz, pauliz));
apply(cnot, np.kron(paulix, pauliz), np.kron(-pauliy, pauliy));
apply(cnot, np.kron(pauliy, pauliz), np.kron(paulix, pauliy));
apply(cnot, np.kron(pauliz, pauliz), np.kron(np.identity(2), pauliz));

apply(cnot, np.kron(pauliz, paulix), np.kron(pauliz, paulix));

[[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]
True
[[ 0  0  0  1]
 [ 0  0 -1  0]
 [ 0 -1  0  0]
 [ 1  0  0  0]]
True
[[ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j  0.-1.j  0.+0.j  0.+0.j]
 [ 0.+1.j  0.+0.j  0.+0.j  0.+0.j]]
True
[[ 1  0  0  0]
 [ 0 -1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]
True
[[ 0  1  0  0]
 [ 1  0  0  0]
 [ 0  0  0 -1]
 [ 0  0 -1  0]]
True


In [130]:
# phase gate. r bit flip when both x and z are 1, i.e. Y -> NEGATIVE X, -Y -> NEGATIVE (-X)
characterise(phase)
# it is like -X2, Y2, X2
characterise(C1_gens[C1_names.index('-X2, Y2, X2')])

X ->  Y
Y ->  -X
Z ->  Z
X ->  Y
Y ->  -X
Z ->  Z


array([[1, 1, 0],
       [1, 0, 1],
       [0, 1, 0]])

In [129]:
# hadamard. r bit flip when both x and z are 1, i.e. Y -> NEGATIVE Y, -Y -> NEGATIVE (-Y)
characterise(hadamard)
# it is like X, -Y2
characterise(C1_gens[C1_names.index('X, -Y2')])

X ->  Z
Y ->  -Y
Z ->  X
X ->  Z
Y ->  -Y
Z ->  X


array([[0, 1, 0],
       [1, 1, 1],
       [1, 0, 0]])

# Old simulation method: creates one of 11520 two-qubit cliffords each time 

In [105]:
def CZ(tab, a, b):
    n = np.shape(tab)[1] // 2
    xa = a
    xb = b
    za = n+a
    zb = n+b
    tab[:,za] = (tab[:,za] + tab[:,xb]) % 2
    tab[:,zb] = (tab[:,zb] + tab[:,xa]) % 2
    
def C1(tab, a, gate_ind):
    n = np.shape(tab)[1] // 2
    xa = a
    za = n+a
    xs = np.copy(tab[:,xa])
    zs = np.copy(tab[:,za])
    r = 2*n
    trans = C1_trans[gate_ind]
    tab[:,r] = (tab[:,r] + 
                xs * ((zs + 1) % 2) * trans[0,2] + 
                xs * zs * trans[1,2] + 
                ((xs + 1) % 2) * zs * trans[2,2]) % 2
    tab[:,xa] = (xs * trans[0,0] + zs * trans[2,0]) % 2
    tab[:,za] = (xs * trans[0,1] + zs * trans[2,1]) % 2
    
def singleClass(tab, a, b):
    C1(tab, a, choice(np.arange(24)))
    C1(tab, b, choice(np.arange(24)))

def cnotClass(tab, a, b):
    C1(tab, a, choice(np.arange(24)))
    C1(tab, b, choice(np.arange(24)))
    CZ(tab, a, b)
    C1(tab, a, choice(S1_ind))
    C1(tab, b, choice(S1y_ind))
    
def iswapClass(tab, a, b):
    C1(tab, a, choice(np.arange(24)))
    C1(tab, b, choice(np.arange(24)))
    CZ(tab, a, b)
    C1(tab, a, C1_names.index('Y2'))
    C1(tab, b, C1_names.index('-X2'))
    CZ(tab, a, b)
    C1(tab, a, choice(S1y_ind))
    C1(tab, b, choice(S1x_ind))
    
def swapClass(tab, a, b):
    C1(tab, a, choice(np.arange(24)))
    C1(tab, b, choice(np.arange(24)))
    CZ(tab, a, 
    C1(tab, a, C1_names.index('-Y2'))
    C1(tab, b, C1_names.index('Y2'))
    CZ(tab, a, b)
    C1(tab, a, C1_names.index('Y2'))
    C1(tab, b, C1_names.index('-Y2'))
    CZ(tab, a, b)
    C1(tab, b, C1_names.index('Y2'))

# random two-qubit Clifford gate applied to a,b
# single qubit class has 576, CNOT class has 5184, iSWAP class 5184, SWAP class has 576, total: 11520
def randGate(tab, a, b):
    prob_distrib = np.array([576, 5184, 5184, 576])/11520
    draw = choice(np.arange(4), p=prob_distrib)
    if draw == 0:
        singleClass(tab, a, b)
    elif draw == 1:
        cnotClass(tab, a, b)
    elif draw == 2:
        iswapClass(tab, a, b)
    elif draw == 3:
        swapClass(tab, a, b)
    else:
        raise Exception('randGate invalid class') 

In [22]:
# for debugging, define some deterministic gates of each class

def singleClassDet(tab, a, b, ind1, ind2):
    if (ind1 < 0) or (ind1 > 23) or (ind2 < 0) or (ind2 > 23):
        raise Exception('invalid inputs')
    C1(tab, a, ind1)
    C1(tab, b, ind2)

def swapClassDet(tab, a, b, ind1, ind2):
    if (ind1 < 0) or (ind1 > 23) or (ind2 < 0) or (ind2 > 23):
        raise Exception('invalid inputs')
    C1(tab, a, ind1)
    C1(tab, b, ind2)
    CZ(tab, a, b)
    C1(tab, a, C1_names.index('-Y2'))
    C1(tab, b, C1_names.index('Y2'))
    CZ(tab, a, b)
    C1(tab, a, C1_names.index('Y2'))
    C1(tab, b, C1_names.index('-Y2'))
    CZ(tab, a, b)
    C1(tab, b, C1_names.index('Y2'))


tab = initTab(2)
print(tab)
print()
singleClassDet(tab, 0, 1, 3, 5)
print(tab)
print()
singleClassDet(tab, 0, 1, 3, 5)
print(tab)
print()


NameError: name 'initTab' is not defined

In [ ]:
# random two-qubit Clifford gate applied to a,b
# single qubit class has 576, CNOT class has 5184, iSWAP class 5184, SWAP class has 576, total: 11520

def randGate(tab, a, b):
    prob_distrib = np.array([576, 5184, 5184, 576])/11520
    draw = np.random.choice(np.arange(4), p=prob_distrib)
    
    i = choice(np.arange(24))
    j = choice(np.arange(24))
    k = None
    l = None

    if draw == 0: # single qubit class
        C1(tab,a,i)
        C1(tab,b,j)
    elif draw == 1: # cnot class
        k = choice(S1_ind)
        l = choice(S1y_ind)
        C1(tab, a, i)
        C1(tab, b, j)
        CZ(tab, a, b)
        C1(tab, a, k)
        C1(tab, b, j)
    elif draw == 2: # iswap class
        k = choice(S1y_ind)
        l = choice(S1x_ind)
        C1(tab, a, i)
        C1(tab, b, j)
        CZ(tab, a, b)
        C1(tab, a, C1_names.index('Y2'))
        C1(tab, b, C1_names.index('-X2'))
        CZ(tab, a, b)
        C1(tab, a, k)
        C1(tab, b, l)
    elif draw == 3: # swap class
        C1(tab, a, i)
        C1(tab, b, j)
        CZ(tab, a, b)
        C1(tab, a, C1_names.index('-Y2'))
        C1(tab, b, C1_names.index('Y2'))
        CZ(tab, a, b)
        C1(tab, a, C1_names.index('Y2'))
        C1(tab, b, C1_names.index('-Y2'))
        CZ(tab, a, b)
        C1(tab, b, C1_names.index('Y2'))
    else:
        raise Exception('randGate invalid class') 
    return draw,i,j,k,l

# Not needed for this application

In [8]:
# c, h, p generator gates by simplified tableau rules (not necessary)

cnot = np.matrix([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
hadamard = 1/np.sqrt(2) * np.matrix([[1,1],[1,-1]])
phase = np.matrix([[1,0],[0,1j]])
swap = np.matrix([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
iswap = np.matrix([[1,0,0,0],[0,0,1j,0],[0,1j,0,0],[0,0,0,1]])

In [ ]:
def c_gate(tab, a, b):
    n = np.shape(tab)[1] // 2
    xa = a
    xb = b
    za = n+a
    zb = n+b
    r = 2*n
    tab[:,r] = (tab[:,r] + tab[:,xa] * tab[:,zb] * ((tab[:,xb] + tab[:,za] + 1) % 2)) % 2
    tab[:,xb] = (tab[:,xb] + tab[:,xa]) % 2
    tab[:,za] = (tab[:,za] + tab[:,zb]) % 2

    
def h_gate(tab, a):
    n = np.shape(tab)[1] // 2
    xa = a
    za = n+a
    r = 2*n
    tab[:,r] = (tab[:,r] + (tab[:,xa] * tab[:,za])) % 2
    temp = np.copy(tab[:, xa])
    tab[:,xa] = tab[:, za]
    tab[:,za] = temp

    
def p_gate(tab, a):
    n = np.shape(tab)[1] // 2
    xa = a
    za = n+a
    r = 2*n
    tab[:,r] = (tab[:,r] + (tab[:,xa] * tab[:,za])) % 2
    tab[:,za] = (tab[:,za] + tab[:,xa]) % 2